![alt text](./pageheader_rose2_babies.jpg)

# Analysis of leak compensation data

#### Author: Dr Gusztav Belteki

This notebook compares recordings **with or without the leak compensation mode** enabled. It imports the *slow_measurements_sippv_vg.pickle* pickle archive generated by the **SIPPV_VG.ipynb** notebook. It only keeps recording **DG050 <= DG050**. Exploratory data analysis and preprocessing is then performed on the imported data. Thereafter, it calculates descriptive statistics and graphs for both groups (leak compensation on or off) and saves them as Excel files and images. It then compares the two groups using inferential statistics. It also generates the *slow_measurement_VG_selected_pars* pickle archive which can be inspected for VTimand analysis and the *low_VTimand* pickle archive which has the extreme cases of VTimand < VTemand. 


**Preprocessing steps performed in this notebook**

*   Only considers DG001-DG050
*   Removed rows with missing VTimand, VTmand, VTemand, leak% or PIP data
*   Removed those data points where VTimand_kg, VTmand_kg or VTemand_kg > 20 mL/kg

### Importing the required libraries and setting options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import re
import pickle

from scipy import stats
from pandas import Series, DataFrame
from datetime import datetime, timedelta

%matplotlib inline

matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'
plt.rcParams['axes.grid'] = True

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
print("Python version: {}".format(sys.version))
print("IPython version: {}".format(IPython.__version__))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))

### Import custom functions from own module

In [ ]:
from gb_loader import *
from gb_stats import *
from gb_transform import *
from gb_visualizer import *

### List and set the working directory and the directories to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'leak_comp'

# Source of the serialized data imported into this Notebook
SOURCE = 'VG'

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Directory containing clinical and blood gas data
CWD = '/Users/guszti/ventilation_draeger'

# Serialised data will be read from this folder of an external hard drive
DIR_READ_1 = '/Volumes/%s/data_dump/draeger/%s' % (DRIVE, SOURCE)

# Alarm data will be read from this folder of the internal hard drive 
DIR_READ_2 = '/Volumes/%s/Draeger/service_evaluation_old' % DRIVE

# Directory to write results and selected images to 
if not os.path.isdir('%s/%s/%s' % (CWD, 'Analyses', TOPIC)):
    os.makedirs('%s/%s/%s' % (CWD, 'Analyses', TOPIC))
DIR_WRITE = '%s/%s/%s' % (CWD, 'Analyses', TOPIC)

# Images and raw data will be written on an external hard drive
if not os.path.isdir('/Volumes/%s/data_dump/draeger/%s' % (DRIVE, TOPIC)):
    os.makedirs('/Volumes/%s/data_dump/draeger/%s' % (DRIVE, TOPIC))
DATA_DUMP = '/Volumes/%s/data_dump/draeger/%s' % (DRIVE, TOPIC)

In [ ]:
os.chdir(CWD)
os.getcwd()

In [ ]:
DIR_READ_1

In [ ]:
DIR_READ_2

In [ ]:
DIR_WRITE

In [ ]:
DATA_DUMP

____

# 1. Ventilator data

### Import ventilator data 
Files ending with **`slow_measurement.csv`** contain ventilator data obtained with 1Hz (1/sec) sampling rate.

Import processed `slow_measurements` data from pickle archive .This has been exported from *'Volume_guarantee_for_paper.opynb'*

In [ ]:
with open('%s/%s.pickle' % (DIR_READ_1, 'slow_measurements_sippv_vg_1'), 'rb') as handle:
    slow_measurements_1 = pickle.load(handle)
with open('%s/%s.pickle' % (DIR_READ_1, 'slow_measurements_sippv_vg_2'), 'rb') as handle:
    slow_measurements_2 = pickle.load(handle)

### Combine dictionaries into one dictionary and delete the subdictionaries to save memory

In [ ]:
slow_measurements = {}
slow_measurements.update(slow_measurements_1)
slow_measurements.update(slow_measurements_2)

In [ ]:
del slow_measurements_1, slow_measurements_2,

In [ ]:
len(slow_measurements)

In [ ]:
recordings = sorted(slow_measurements.keys())

In [ ]:
print(recordings)

### Keep recordings <=50

In [ ]:
to_remove = ['DG051_1', 'DG052', 'DG054', 'DG055', 'DG056', 'DG057', 'DG059', 'DG060']

for recording in to_remove:
    slow_measurements.pop(recording)

recordings = sorted(slow_measurements.keys())
print(recordings)

### Identify which recordings were leak-compensated and which were not

**'leak_comp'** = **VTmand_kg** - **VTemand_kg**

*  If the leak-compensation if off VTmand = VTemand. The targeted parameter is VTemand.
*  If leak-compesation is on, VTmand > VTemand. The targeted parameter is VTmand



In [ ]:
leak_comp_on = []
leak_comp_off = []

for recording in recordings:
    if slow_measurements[recording]['leak_comp'].mean() > 0.001:
        leak_comp_on.append(recording) 
    else:
        leak_comp_off.append(recording) 

In [ ]:
print(leak_comp_on)

In [ ]:
print('Leak compensation was ON for %d recordings' % len(leak_comp_on))

In [ ]:
print(leak_comp_off)

In [ ]:
print('Leak compensation was OFF for %d recordings' % len(leak_comp_off))

____

# 2. Clinical data

### Import clinical details

In [ ]:
clinical_details = pd.read_excel('%s/data_grabber_patient_data_combined_OLD.xlsx' % CWD)
clinical_details.index = clinical_details['Recording']

In [ ]:
clinical_details = clinical_details.reindex(recordings)

In [ ]:
clinical_details['Recording start'] = pd.to_datetime(clinical_details['Recording period'].apply(lambda x: x[:10]),
                dayfirst = True)
clinical_details['Postnatal age'] = clinical_details['Recording start'] - clinical_details['Date of birth']
clinical_details['Corrected gestation'] = clinical_details['Gestation'] + \
                clinical_details['Postnatal age'].astype(int) / (1E+9 * 3600 * 24 * 7)

In [ ]:
clinical_details.info()

In [ ]:
current_weights = {}
for recording in recordings:
    current_weights[recording] = clinical_details.loc[recording, 'Current weight' ] / 1000

In [ ]:
clinical_details;

### Statistics about clinical details

In [ ]:
clinical_details.loc[leak_comp_on];

In [ ]:
clinical_details.loc[leak_comp_off];

In [ ]:
# Gestation does not show normal distribution
fig, axes = plt.subplots(1,2, figsize = (6,3), sharey = True)

axes[0].hist(clinical_details.loc[leak_comp_on]['Gestation'], 
             bins = 10, rwidth = 0.8, color = 'k')

axes[1].hist(clinical_details.loc[leak_comp_off]['Gestation'], 
             bins = 10, rwidth = 0.8, color = 'k')
axes[0].set_xlabel('weeks of gestation'); axes[1].set_xlabel('weeks of gestation')
axes[0].set_title('Leak compensation ON', size = 12); axes[1].set_title('Leak compensation OFF', size = 12)
axes[0].set_ylabel('number of patients'); axes[1].set_ylabel('number of patients')
axes[1].yaxis.tick_right(); axes[1].yaxis.set_label_position("right")

plt.tight_layout()

In [ ]:
# Corrected gestation does not show normal distribution
fig, axes = plt.subplots(1,2, figsize = (6,3), sharey = True)

axes[0].hist(clinical_details.loc[leak_comp_on]['Corrected gestation'], 
             bins = 10, rwidth = 0.8, color = 'k')

axes[1].hist(clinical_details.loc[leak_comp_off]['Gestation'], 
             bins = 10, rwidth = 0.8, color = 'k')
axes[0].set_xlabel('corrected gestation (weeks)'); axes[1].set_xlabel('corrected gestation (weeks)')
axes[0].set_title('Leak compensation ON', size = 12); axes[1].set_title('Leak compensation OFF', size = 12)
axes[0].set_ylabel('number of patients'); axes[1].set_ylabel('number of patients')
axes[1].yaxis.tick_right(); axes[1].yaxis.set_label_position("right")

plt.tight_layout()

In [ ]:
# Current weight does not show normal distribution
fig, axes = plt.subplots(1,2, figsize = (12,3), sharey = True)
axes[0].hist(clinical_details.loc[leak_comp_on]['Current weight'], 
             bins = 10, rwidth = 0.8, color = 'k')
axes[1].hist(clinical_details.loc[leak_comp_off]['Current weight'], 
             bins = 10, rwidth = 0.8, color = 'k');

In [ ]:
pars = ['Gestation', 'Corrected gestation', 'Birth weight', 'Current weight']

clin_details_stats_lc = round(DataFrame([clinical_details.loc[leak_comp_on][pars].median(),
                                     clinical_details.loc[leak_comp_on][pars].min(),
                                     clinical_details.loc[leak_comp_on][pars].max()]).T, 2)
clin_details_stats_lc.columns = ['median', 'min', 'max']


In [ ]:
pars = ['Gestation', 'Corrected gestation', 'Birth weight', 'Current weight']

clin_details_stats_no_lc = round(DataFrame([clinical_details.loc[leak_comp_off][pars].median(),
                                     clinical_details.loc[leak_comp_off][pars].min(),
                                     clinical_details.loc[leak_comp_off][pars].max()]).T, 2)
clin_details_stats_no_lc.columns = ['median', 'min', 'max']


In [ ]:
clin_details_stats = pd.concat([clin_details_stats_lc, clin_details_stats_no_lc], axis = 1,
                              keys = ['Leak compensation ON', 'Leak compensation OFF'])

In [ ]:
mann_whitney_stat = []
mann_whitney_p = []
for par in pars:
    mann_whitney_stat.append(stats.mannwhitneyu(clinical_details.loc[leak_comp_on][par], 
                                        clinical_details.loc[leak_comp_off][par])[0])
    mann_whitney_p.append(stats.mannwhitneyu(clinical_details.loc[leak_comp_on][par], 
                                        clinical_details.loc[leak_comp_off][par])[1])
# clin_details_stats['Mann_Whitney stat'] = mann_whitney_stat
clin_details_stats['Mann_Whitney p'] = mann_whitney_p

In [ ]:
ttest_stat = []
ttest_p = []
for par in pars:
    ttest_stat.append(stats.ttest_ind(clinical_details.loc[leak_comp_on][par], 
                                        clinical_details.loc[leak_comp_off][par])[0])
    ttest_p.append(stats.ttest_ind(clinical_details.loc[leak_comp_on][par], 
                                        clinical_details.loc[leak_comp_off][par])[1])
# clin_details_stats['Mann_Whitney stat'] = mann_whitney_stat
clin_details_stats['ttest'] = ttest_p

There are no significant difference in clinical parameters

In [ ]:
clin_details_stats

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'clinical_details_leak_comp_group_comp.xlsx'))
clinical_details.loc[leak_comp_on].to_excel(writer, 'leak_comp_on')
clinical_details.loc[leak_comp_off].to_excel(writer, 'leak_comp_off')
clin_details_stats.to_excel(writer, 'group_stats')
writer.save()

____

# 3. Preprocessing of ventilator data

### Combine `slow_measurements` DataFrames into one

In [ ]:
total = []
for recording in recordings:
    total.append(slow_measurements[recording])
slow_measurements_all = pd.concat(total)    

In [ ]:
# Total number of seconds as well
len(slow_measurements_all)

### Missing data

In [ ]:
# How many percent of points were missing for the different parameters?

missing = slow_measurements_all.isnull().sum()
missing_pc = round((missing / len(slow_measurements_all)) * 100, 3)
missing_pc.sort_values()

### Remove rows with missing VTimand, VTmand, VTemand, leak% or PIP data

In [ ]:
a = len(slow_measurements_all)
print('Before removal: %d rows' % a)

slow_measurements_all.dropna(axis = 0, subset = ['VTimand_kg', 'VTmand_kg', 
                                                 'VTemand_kg', 'leak%', 'PIP'], inplace = True)

b = len(slow_measurements_all)
print('After removal: %d rows' % b)
print('Removed %d rows' % (a-b))

In [ ]:
len(slow_measurements_all[slow_measurements_all['leak_comp_ON'] == 0])

In [ ]:
len(slow_measurements_all[slow_measurements_all['leak_comp_ON'] == 1])

In [ ]:
# How many percent of points are missing for the different parameters?
print('How many percent of points are missing for the different parameters?')
missing = slow_measurements_all.isnull().sum()
missing_pc = round((missing / len(slow_measurements_all)) * 100, 3)
missing_pc.sort_values()

### Remove those rows where  VTmand_kg or VTemand_kg > 20 mL/kg as these seem to be outliers rather than reflecting real effective or expiratory tidal volumes

In [ ]:
a = len(slow_measurements_all)
print('Before removal: %d rows' % a)
 
slow_measurements_all = slow_measurements_all[slow_measurements_all['VTmand_kg'] <= 20]
slow_measurements_all = slow_measurements_all[slow_measurements_all['VTemand_kg'] <= 20]

b = len(slow_measurements_all)
print('After removal: %d rows' % b)
print('Removed %d rows' % (a-b))

In [ ]:
len(slow_measurements_all[slow_measurements_all['leak_comp_ON'] == 0])

In [ ]:
len(slow_measurements_all[slow_measurements_all['leak_comp_ON'] == 1])

### Create separate DataFrames with *leak-compensated* and *non-leak-compensated* recordings

In [ ]:
slow_measurements_all_lc = \
    slow_measurements_all[slow_measurements_all['recording'].isin(leak_comp_on)].copy()
    
slow_measurements_all_no_lc = \
    slow_measurements_all[slow_measurements_all['recording'].isin(leak_comp_off)].copy()

In [ ]:
print('Number of seconds in leak-compensated recordings: %d' 
      % len(slow_measurements_all_lc))
print('Number of seconds in non-leak-compensated recordings: %d' 
      % len(slow_measurements_all_no_lc))

___

# 4. What was the volume of leak compensation?

### VTmand_kg < VTemand_kg ("negative leak_comp")
How many times was the leak compensated tidal volume inappropriate ( lower than VTemand)?

Only consider **leak-compensated** recordings as otherwise VTmand = VTemand by definition

In [ ]:
a = (len(slow_measurements_all_lc[slow_measurements_all_lc['leak_comp'] < 0]) / 
    len(slow_measurements_all_lc)) * 100
print('Percentage of breaths in LEAK-COMPENSATED breaths when VTmand < VTemand is %.2f' % a)

### Remove rows when leak compensation was negative

In [ ]:
a = len(slow_measurements_all_lc)
print('Before removal: %d rows' % a)
 
slow_measurements_all_lc = slow_measurements_all_lc[slow_measurements_all_lc['leak_comp'] >= 0]

b = len(slow_measurements_all_lc)
print('After removal: %d rows' % b)
print('Removed %d rows' % (a-b))

### How frequently was VTmand_kg = VTemand_kg:  leak compensation  = zero

Only consider **leak-compensated** recordings as otherwise VTmand = VTemand by definition

In [ ]:
a = (len(slow_measurements_all_lc[slow_measurements_all_lc['leak_comp'] == 0]) / 
    len(slow_measurements_all_lc)) * 100
print('Percentage of breaths in LEAK-COMPENSATED breaths when VTmand = VTemand is %.2f' % a)

### How frequently was leak compensation  <= 1 mL/kg (but not zero)

Only consider **leak-compensated** recordings as otherwise VTmand = VTemand by definition

In [ ]:
a = (len(slow_measurements_all_lc[slow_measurements_all_lc['leak_comp'] < 1]) / 
    len(slow_measurements_all_lc)) * 100
print('%.2f' % a)

In [ ]:
a = (len(slow_measurements_all_lc[slow_measurements_all_lc['leak_comp'] < 2]) / 
    len(slow_measurements_all_lc)) * 100
print('%.2f' % a)

### Distribution of leak compensation volume

In [ ]:
cats_leak_comp = pd.cut(slow_measurements_all_lc['leak_comp'], bins = range(0, 22, 1),
                        right = True)
cats_leak_comp.value_counts().sort_index()

Most frequently the leak compensation was between 0-4 mL/kg. What percentage?

In [ ]:
(cats_leak_comp.value_counts() / len(cats_leak_comp)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_leak_comp.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'leak compensation', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of breaths', size = 14)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_comp_1_lin', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_leak_comp.value_counts().sort_index().plot(kind = 'bar', logy = True, 
                        title = 'leak compensation', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of breaths', size = 14)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_comp_1_log', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

#### Zoom in to range [0, 4]

In [ ]:
cats_leak_comp_zoom = pd.cut(slow_measurements_all_lc['leak_comp'], bins = range(5),
                        right = False)
cats_leak_comp_zoom.value_counts().sort_index()

In [ ]:
(cats_leak_comp_zoom.value_counts() / len(cats_leak_comp)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_leak_comp_zoom.value_counts().sort_index().plot(kind = 'bar', logy = True, 
                        title = 'leak compensation', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of breaths', size = 14)
ax.set_ylim(1, 10000000)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_comp_2', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

#### Zoom in to range [0, 1]

In [ ]:
cats_leak_comp_zoom2 = pd.cut(slow_measurements_all_lc['leak_comp'], 
                              bins = np.arange(0, 1.2, 0.2), right = False)
cats_leak_comp_zoom2.value_counts().sort_index()

In [ ]:
(cats_leak_comp_zoom2.value_counts() / len(cats_leak_comp)) * 100

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_leak_comp_zoom2.value_counts().sort_index().plot(kind = 'bar', logy = True, 
                        title = 'leak compensation', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of breaths', size = 14)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_comp_3', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

#### Combined

In [ ]:
bins = [ 0, 0.2, 0.4, 0.6, 0.8, 1, 2, 3, 4, 8, 12, 16, 20]

cats_leak_comp_custom = pd.cut(slow_measurements_all_lc['leak_comp'], 
                              bins = bins, right = False)
cats_leak_comp_custom.value_counts().sort_index()

In [ ]:
(cats_leak_comp_custom.value_counts() / len(cats_leak_comp)) * 100

In [ ]:
# This figure has got semilogarithmic scale

fig, ax = plt.subplots(figsize = [8,6])
cats_leak_comp_custom.value_counts().sort_index().plot(kind = 'bar', logy = True, 
                        title = 'leak compensation', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of breaths', size = 14)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_comp_4_log', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
# This figure has got linear scale

fig, ax = plt.subplots(figsize = [8,6])
cats_leak_comp_custom.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'leak compensation', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of breaths', size = 14)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_comp_4_lin', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

___

# 5. Analyse and compare leak-compensated and non-leak compensated inflations

In [ ]:
# Consider the following parameters only:  
select_pars_2 = ['leak_comp', 'leak%', 'VTimand_kg',  'VTmand_kg',
                 'VTemand_kg',  'VTset_kg', 'VT_diff', 'RR', 'RR_set', 'RRmand', 'MV_kg', 'MVemand_kg', 
                 'MVleak_kg', 'PIP', 'Pmean', 'PEEP', 'P_diff', 'FiO2']

### Statistics of parameters in case of leak compensation on and off 

In [ ]:
# Name of descriptive statistics calculated
stat_list = ['seconds', 'mean', 'SD', 'min', '5th pc', 
             '25th pc', 'median', '75th pc', '95th pc', 'max']

In [ ]:
stats_all_lc = {}

for par in select_pars_2:
    stats_all_lc[par] = \
        round(slow_measurements_all_lc[par].describe(percentiles = [0.05, 0.25, 0.5, 0.75, 0.95]), 2)
    stats_all_lc[par].index = stat_list
    
stats_all_lc = DataFrame(stats_all_lc).T

In [ ]:
stats_all_lc

In [ ]:
stats_all_no_lc = {}

for par in select_pars_2:
    stats_all_no_lc[par] = \
        round(slow_measurements_all_no_lc[par].describe(percentiles = [0.05, 0.25, 0.5, 0.75, 0.95]), 2)
    stats_all_no_lc[par].index = stat_list
    
stats_all_no_lc = DataFrame(stats_all_no_lc).T

In [ ]:
stats_all_no_lc

It is not meaningful to perform T-tests to compare these groups. Due to the very large size of the groups (>1 million) even small differences would be statistically highly significant.

In [ ]:
stats_all = pd.concat([stats_all_lc[['mean', 'median', 'SD']],
                       stats_all_no_lc[['mean', 'median', 'SD']]], axis = 1, 
                       keys = ['leak compensation ON', 'leak compensation OFF'])
stats_all           

For **Leak%, leak_comp, MVleak and VT_diff** the mean and median are very different, these are not normally distributed. For these use non-parametric statistics (median, IQR). For the rest use parametric tests (mean, SD).

In [ ]:
# Save stats to Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_all_lc_no_lc.xlsx'))
stats_all_lc.to_excel(writer, 'leak_comp_on' )
stats_all_no_lc.to_excel(writer, 'leak_comp_off')
stats_all.to_excel(writer, 'comparison')
writer.save()

### Visualize and compare the distribution of the some of the parameters in the leak-compensation on and off groups using histograms

In [ ]:
# VTimand_kg 

bins = 20
fig, axes = plt.subplots(1,2, sharey = False, figsize = (7,3))
axes[0].hist(slow_measurements_all_lc['VTimand_kg'], bins = np.arange(0,21,1), color = 'black', alpha = 0.75);
axes[1].hist(slow_measurements_all_no_lc['VTimand_kg'], bins = np.arange(0,21,1), color = 'black', alpha = 0.75)
axes[0].set_ylabel('number of breaths'); axes[1].set_ylabel('number of breaths')
axes[0].set_title('Leak compensation ON', size = 12); axes[1].set_title('Leak compensation OFF', size = 12)
axes[0].set_xlabel('VTimand (mL/kg)'); axes[1].set_xlabel('VTimand (mL/kg)')
axes[1].yaxis.tick_right(); axes[1].yaxis.set_label_position("right")
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTimand_kg', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
# VTmand_kg (this is the parameter which is targeted to the set VT; in case of leak compensation
# the leak-compensated VT, without leak compensation, the expiratory VT (VTemand)

bins = 20
fig, axes = plt.subplots(1,2, sharey = False, figsize = (7,3))
axes[0].hist(slow_measurements_all_lc['VTmand_kg'], bins = np.arange(0,21,1), color = 'black', alpha = 0.75);
axes[1].hist(slow_measurements_all_no_lc['VTmand_kg'], bins = np.arange(0,21,1), color = 'black', alpha = 0.75)
axes[0].set_ylabel('number of breaths'); axes[1].set_ylabel('number of breaths')
axes[0].set_title('Leak compensation ON', size = 12); axes[1].set_title('Leak compensation OFF', size = 12)
axes[0].set_xlabel('VTmand (mL/kg)'); axes[1].set_xlabel('VTmand (mL/kg)')
axes[1].yaxis.tick_right(); axes[1].yaxis.set_label_position("right")
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTmand_kg', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
# VTemand_kg 

bins = 20
fig, axes = plt.subplots(1,2, sharey = False, figsize = (7,3))
axes[0].hist(slow_measurements_all_lc['VTemand_kg'], bins = np.arange(0,21,1), color = 'black', alpha = 0.75);
axes[1].hist(slow_measurements_all_no_lc['VTemand_kg'], bins = np.arange(0,21,1), color = 'black', alpha = 0.75)
axes[0].set_ylabel('number of breaths'); axes[1].set_ylabel('number of breaths')
axes[0].set_title('Leak compensation ON', size = 12); axes[1].set_title('Leak compensation OFF', size = 12)
axes[0].set_xlabel('VTemand (mL/kg)'); axes[1].set_xlabel('VTemand (mL/kg)')
axes[1].yaxis.tick_right(); axes[1].yaxis.set_label_position("right")
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VTemand_kg', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
# VT_diff = VTmand_kg - VTset_kg 

bins = 20
fig, axes = plt.subplots(1,2, sharey = False, figsize = (7,3))
axes[0].hist(slow_measurements_all_lc.dropna()['VT_diff'], 
             bins = np.arange(-20, 20, 1), color = 'black', alpha = 0.75);
axes[1].hist(slow_measurements_all_no_lc.dropna()['VT_diff'], 
             bins = np.arange(-20, 20, 1), color = 'black', alpha = 0.75)
axes[0].set_ylabel('number of breaths'); axes[1].set_ylabel('number of breaths')
axes[0].set_title('Leak compensation ON', size = 12); axes[1].set_title('Leak compensation OFF', size = 12)
axes[0].set_xlabel('VT_diff (mL/kg)'); axes[1].set_xlabel('VT_diff (mL/kg)')
axes[1].yaxis.tick_right(); axes[1].yaxis.set_label_position("right")
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'VT_diff', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

#### How frequently (%) was VT_diff < 0 (VTmand < VT_set) ?

In [ ]:
len(slow_measurements_all_lc[slow_measurements_all_lc['VT_diff'] <0]) / len(slow_measurements_all_lc) * 100

In [ ]:
len(slow_measurements_all_no_lc[slow_measurements_all_no_lc['VT_diff'] <0]) / len(slow_measurements_all_no_lc) * 100

In [ ]:
# MV_kg

bins = 20
fig, axes = plt.subplots(1,2, sharey = False, figsize = (7,3))
axes[0].hist(slow_measurements_all_lc.dropna()['MV_kg'], bins = np.arange(0, 1, 0.05),  color = 'black', alpha = 0.75);
axes[1].hist(slow_measurements_all_no_lc.dropna()['MV_kg'], bins = np.arange(0, 1, 0.05), color = 'black', alpha = 0.75)
axes[0].set_ylabel('number of breaths'); axes[1].set_ylabel('number of breaths')
axes[0].set_title('Leak compensation ON', size = 12); axes[1].set_title('Leak compensation OFF', size = 12)
axes[0].set_xlabel('MV (L/min/kg)'); axes[1].set_xlabel('MV (L/min/kg)')
axes[1].yaxis.tick_right(); axes[1].yaxis.set_label_position("right")
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'MV_kg', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
# RRmand

bins = 20
fig, axes = plt.subplots(1,2, sharey = False, figsize = (7,3))
axes[0].hist(slow_measurements_all_lc.dropna()['RRmand'], 
             bins = np.arange(0, 100, 5),  color = 'black', alpha = 0.75);
axes[1].hist(slow_measurements_all_no_lc.dropna()['RRmand'], 
             bins = np.arange(0, 100, 5), color = 'black', alpha = 0.75)
axes[0].set_ylabel('number of breaths'); axes[1].set_ylabel('number of breaths')
axes[0].set_title('Leak compensation ON', size = 12); axes[1].set_title('Leak compensation OFF', size = 12)
axes[0].set_xlabel('RR (breaths/min)'); axes[1].set_xlabel('RR (breaths/min)')
axes[1].yaxis.tick_right(); axes[1].yaxis.set_label_position("right")
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'RRmand', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
# PIP

bins = 20
fig, axes = plt.subplots(1,2, sharey = False, figsize = (7,3))
axes[0].hist(slow_measurements_all_lc.dropna()['PIP'], 
             bins = np.arange(0, 50, 2.5),  color = 'black', alpha = 0.75);
axes[1].hist(slow_measurements_all_no_lc.dropna()['PIP'], 
             bins = np.arange(0, 50, 2.5), color = 'black', alpha = 0.75)
axes[0].set_ylabel('number of breaths'); axes[1].set_ylabel('number of breaths')
axes[0].set_title('Leak compensation ON', size = 12); axes[1].set_title('Leak compensation OFF', size = 12)
axes[0].set_xlabel('PIP (mbar)'); axes[1].set_xlabel('PIP (mbar)')
axes[1].yaxis.tick_right(); axes[1].yaxis.set_label_position("right")
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'PIP', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
# P_diff (= Pmax - PIP)

bins = 20
fig, axes = plt.subplots(1,2, sharey = False, figsize = (7,3))
axes[0].hist(slow_measurements_all_lc.dropna()['P_diff'], 
             bins = np.arange(-10, 50, 1),  color = 'black', alpha = 0.75);
axes[1].hist(slow_measurements_all_no_lc.dropna()['P_diff'], 
             bins = np.arange(-10, 50, 1), color = 'black', alpha = 0.75)
axes[0].set_ylabel('number of breaths'); axes[1].set_ylabel('number of breaths')
axes[0].set_title('Leak compensation ON', size = 12); axes[1].set_title('Leak compensation OFF', size = 12)
axes[0].set_xlabel('P_diff (mbar)'); axes[1].set_xlabel('P_diff (mbar)')
axes[1].yaxis.tick_right(); axes[1].yaxis.set_label_position("right")
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'P_diff', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
# P_diff (= Pmax - PIP)

bins = 20
fig, axes = plt.subplots(1,2, sharey = False, figsize = (7,3))
axes[0].hist(slow_measurements_all_lc.dropna()['P_diff'], 
             bins = np.arange(-10, 50, 1),  color = 'black', alpha = 0.75)
axes[1].hist(slow_measurements_all_no_lc.dropna()['P_diff'], 
             bins = np.arange(-10, 50, 1), color = 'black', alpha = 0.75)
axes[0].set_ylabel('number of breaths'); axes[1].set_ylabel('number of breaths')
axes[0].set_title('Leak compensation ON', size = 12); axes[1].set_title('Leak compensation OFF', size = 12)
axes[0].set_xlabel('P_diff (mbar)'); axes[1].set_xlabel('P_diff (mbar)')
axes[1].yaxis.tick_right(); axes[1].yaxis.set_label_position("right")
axes[0].set_yscale('log'); axes[1].set_yscale('log')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'P_diff_log', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
# leak%

fig, axes = plt.subplots(1,2, sharey = False, figsize = (7,3))
axes[0].hist(slow_measurements_all_lc.dropna()['leak%'], 
             bins = np.arange(0, 102, 2.5),  color = 'black', alpha = 0.75);
axes[1].hist(slow_measurements_all_no_lc.dropna()['leak%'], 
             bins = np.arange(0, 102, 2.5), color = 'black', alpha = 0.75)
axes[0].set_ylabel('number of breaths'); axes[1].set_ylabel('number of breaths')
axes[0].set_title('Leak compensation ON', size = 12); axes[1].set_title('Leak compensation OFF', size = 12)
axes[0].set_xlabel('Leak (%)'); axes[1].set_xlabel('Leak (%)')
axes[1].yaxis.tick_right(); axes[1].yaxis.set_label_position("right")
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_pc', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
# leak%

fig, axes = plt.subplots(1,2, sharey = False, figsize = (7,3))
axes[0].hist(slow_measurements_all_lc.dropna()['leak%'], 
             bins = np.arange(0, 102, 2.5),  color = 'black', alpha = 0.75);
axes[1].hist(slow_measurements_all_no_lc.dropna()['leak%'], 
             bins = np.arange(0, 102, 2.5), color = 'black', alpha = 0.75)
axes[0].set_ylabel('number of breaths'); axes[1].set_ylabel('number of breaths')
axes[0].set_title('Leak compensation ON', size = 12); axes[1].set_title('Leak compensation OFF', size = 12)
axes[0].set_xlabel('Leak (%)'); axes[1].set_xlabel('Leak (%)')
axes[1].yaxis.tick_right(); axes[1].yaxis.set_label_position("right")
axes[0].set_yscale('log'); axes[1].set_yscale('log')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_pc', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

____

# 6. Visualise the relationship between leak% and some of the ventilator parameters

### Compare leak-compensated and non-leak compensated inflations having various levels of leak

### Create bins to various leak% levels

Create bins for leak and group breaths accordingly.

Bins of **leak percentage**:

*  0 - 10 %
* 10 - 20 %
* 20 - 30 %
* 30  -40 %
* 40 - 50 %
* 50 - 60 %
* 60 - 70 %
* 70 - 80 %
* 80 - 90 %
* 90 - 100%

In [ ]:
# Create bins for leak% every 10%
cats_leak_pc_lc = slow_measurements_all_lc.groupby(pd.cut(slow_measurements_all_lc['leak%'], 
                                np.arange(0, 105, 10), right = False))
cats_leak_pc_lc.size()

In [ ]:
# Create bins for leak% every 10%
cats_leak_pc_no_lc = slow_measurements_all_no_lc.groupby(pd.cut(slow_measurements_all_no_lc['leak%'], 
                                np.arange(0, 105, 10), right = False))
cats_leak_pc_no_lc.size()

In [ ]:
# What was the percentage of inflations with leak of <20%
len(slow_measurements_all_lc[slow_measurements_all_lc['leak%'] < 20]) / len(slow_measurements_all_lc)

### Means of parameters over ranges of leak in case of leak compensation on and off

In [ ]:
select_pars_paper_lc = ['number of inflations', 'leak_comp', 'leak%', 'VTmand_kg', 'VTemand_kg', 'VTset_kg', 'VT_diff', 'RR', 
                         'MV_kg', 'MVleak_kg', 'PIP', 'Pmean', 'P_diff', 'FiO2']

select_pars_paper_no_lc = ['number of inflations', 'leak%', 'VTemand_kg', 'VTset_kg', 'VT_diff', 'RR', 
                         'MV_kg', 'MVleak_kg', 'PIP', 'Pmean', 'P_diff', 'FiO2']

In [ ]:
stats_lc = cats_leak_pc_lc.mean()
stats_lc['number of inflations'] = cats_leak_pc_lc.size()
stats_lc = round(stats_lc, 2)
stats_lc_paper = stats_lc[select_pars_paper_lc]
stats_lc_paper = round(stats_lc_paper, 2)
stats_lc_paper

In [ ]:
stats_no_lc = cats_leak_pc_no_lc.mean()
stats_no_lc['number of inflations'] = cats_leak_pc_no_lc.size()
stats_no_lc = round(stats_no_lc, 2)
stats_no_lc_paper = stats_no_lc[select_pars_paper_no_lc]
stats_no_lc_paper = round(stats_no_lc_paper, 2)
stats_no_lc_paper

In [ ]:
# This manipulation is required as the original index (range) cannot be exported to Excel file
stats_lc_mod = stats_lc_paper.copy()
stats_lc_mod.index = ['0-10', '10_20', '20-30', '30-40', '40-50', '50-60', '60-70',
                      '70-80', '80-90', '90-100']
stats_lc_mod.index.rename('leak range (%)', inplace  = True)

In [ ]:
# This manipulation is required as the original index (range) cannot be exported to Excel file
stats_no_lc_mod = stats_no_lc_paper.copy()
stats_no_lc_mod.index = ['0-10', '10_20', '20-30', '30-40', '40-50', '50-60', '60-70',
                      '70-80', '80-90', '90-100']
stats_no_lc_mod.index.rename('leak range (%)', inplace  = True)

In [ ]:
# Save stats to Excel file
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_lc_no_lc.xlsx'))
stats_lc_mod.to_excel(writer, 'leak_comp_on' )
stats_no_lc_mod.to_excel(writer, 'leak_comp_off')
writer.save()

### Number of inflations for each leak% bin

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_leak_pc_lc.size().plot(kind = 'bar', logy = False, 
                        title = 'Leak compensation ON', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('leak (%)', size = 14)
ax.set_ylabel('number of breaths', size = 14)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_pc_lc_on_lin', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
fig, ax = plt.subplots(figsize = [8,6])
cats_leak_pc_lc.size().plot(kind = 'bar', logy = True, 
                        title = 'Leak compensation ON', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('leak (%)', size = 14)
ax.set_ylabel('number of breaths', size = 14)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_pc_lc_on_log', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

### The volume of leak compensation at different levels of leak

In [ ]:
bins = np.arange(0, 105, 10)
cats_leak_diff_no_lc = pd.cut(slow_measurements_all_no_lc['leak%'], 
                              bins = bins, right = False)
grouped_no_lc = slow_measurements_all_no_lc.groupby(cats_leak_diff_no_lc)

In [ ]:
bins = np.arange(0, 105, 10)
cats_leak_diff_lc = pd.cut(slow_measurements_all_lc['leak%'], 
                              bins = bins, right = False)
grouped_lc = slow_measurements_all_lc.groupby(cats_leak_diff_lc)

In [ ]:
grouped_lc['leak_comp'].describe(percentiles = [0.05, 0.25, 0.50, 0.75, 0.95])

In [ ]:
grouped_no_lc['leak_comp'].describe(percentiles = [0.05, 0.25, 0.50, 0.75, 0.95])

#### Correlation between leak% and leak compensation

In [ ]:
slow_measurements_all_lc[['leak%', 'leak_comp']].corr(method='pearson')

In [ ]:
from scipy.stats import pearsonr

def corr_pearson(x, y):

    '''
    input: two numeric arrays of the same size

    returns: a tuple of 
    1. Pearson's correlation coefficient: r
    2. low and high 95% confidence intervals or r (two values)
    3. Coefficient of determination: r^2
    4. p-value of correlation

    '''
    
    assert len(x) == len(y)
    
    r, p = pearsonr(x, y)
    f = 0.5*np.log((1+r)/(1-r))
    se = 1/np.sqrt(len(x)-3)
    ucl = f + 1.96 * se
    lcl = f - 1.96 * se

    lcl = (np.exp(2*lcl) - 1) / (np.exp(2*lcl) + 1)
    ucl = (np.exp(2*ucl) - 1) / (np.exp(2*ucl) + 1)

    return r , lcl, ucl , r*r, p

In [ ]:
from scipy.stats import spearmanr

def corr_spearman(x, y):

    '''
    input: two numeric arrays of the same size

    returns: a tuple of 
    1. Spearman's correlation coefficient: r
    2. low and high 95% confidence intervals or r (two values)
    3. Coefficient of determination: r^2
    4. p-value of correlation

    '''
    
    assert len(x) == len(y)
    
    r, p = spearmanr(x, y)
    f = 0.5*np.log((1+r)/(1-r))
    se = 1/np.sqrt(len(x)-3)
    ucl = f + 1.96 * se
    lcl = f - 1.96 * se

    lcl = (np.exp(2*lcl) - 1) / (np.exp(2*lcl) + 1)
    ucl = (np.exp(2*ucl) - 1) / (np.exp(2*ucl) + 1)

    return r , lcl, ucl , r*r, p

In [ ]:
r , lcl, ucl , r2, p = corr_pearson(slow_measurements_all_lc['leak%'], 
                                      slow_measurements_all_lc['leak_comp'])
print('r = {0:.2f}, lcl = {1:.2f}, ucl = {2:.2f}, r2 = {3:.2f}, p = {4:.6f}'.format(r , lcl, ucl , r2, p))

In [ ]:
r , lcl, ucl , r2, p = corr_spearman(slow_measurements_all_lc['leak%'], 
                                      slow_measurements_all_lc['leak_comp'])
print('r = {0:.2f}, lcl = {1:.2f}, ucl = {2:.2f}, r2 = {3:.2f}, p = {4:.6f}'.format(r , lcl, ucl , r2, p))

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(figsize = (8,6))
data = []
for name, group in grouped_no_lc:
    data.append(group['leak_comp'].values)
plt.boxplot(data, whis = [5, 95], showfliers = False,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('Mean leak compensation (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_pc_no_lc_leak_comp_boxplot', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(figsize = (8,6))
data = []
for name, group in grouped_lc:
    data.append(group['leak_comp'].values)
plt.boxplot(data, whis = [5, 95], showfliers = False,showmeans = True,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('Leak compensation (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_pc_lc_leak_comp_boxplot', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

### Mean VT_diff for each leak% bin

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(figsize = (8,6))
data = []
for name, group in grouped_no_lc:
    data.append(group['VT_diff'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False, showmeans = True,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('VT_diff (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_pc_no_lc_VT_diff_boxplot', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(figsize = (8,6))
data = []
for name, group in grouped_lc:
    data.append(group['VT_diff'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False, showmeans = True,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('VT_diff (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_pc_lc_VT_diff_boxplot', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

### Mean P_diff (Pmax - PIP) for each leak% bin

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(figsize = (8,6))
data = []
for name, group in grouped_no_lc:
    data.append(group['P_diff'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False, showmeans = True,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('P_diff (mbar)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_pc_no_lc_Pdiff_boxplot', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(figsize = (8,6))
data = []
for name, group in grouped_lc:
    data.append(group['P_diff'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False, showmeans = True,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('P_diff (mbar)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'leak_pc_lc_Pdiff_boxplot', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

____

# 7. Descriptive statistics on individual recordings and paramaters

### Calculate descriptive statistics in each recording for all the parameters 

In [ ]:
# Selected statistics
stat_list = ['seconds', 'mean', 'SD', 'min', '5th pc', 
             '25th pc', 'median', '75th pc', '95th pc', 'max']

In [ ]:
stats_all_recs = {}

for recording in recordings:
    
    stats_all_recs[recording] = \
        round(slow_measurements_all[slow_measurements_all['recording'] == 
        recording].describe(percentiles = [0.05, 0.25, 0.5, 0.75, 0.95]), 2)
    stats_all_recs[recording].index = stat_list
    stats_all_recs[recording] = stats_all_recs[recording].T
    stats_all_recs[recording]['IQR'] = stats_all_recs[recording]['75th pc'] - stats_all_recs[recording]['25th pc']

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_all_rec_lc_on.xlsx'))
for rec in leak_comp_on:
    stats_all_recs[rec].to_excel(writer, rec)
writer.save()

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_all_rec_lc_off.xlsx'))
for rec in leak_comp_off:
    stats_all_recs[rec].to_excel(writer, rec)
writer.save()

### Calculate descriptive statistics in all recordings for each of the parameters 

In [ ]:
recs = slow_measurements_all.groupby('recording')

In [ ]:
stats_all_pars = {}

for par in sorted(slow_measurements_all.keys())[:-1]: # removes the 'recording' field
    stats_all_pars[par] = \
        round(recs[par].describe(percentiles = [0.05, 0.25, 0.5, 0.75, 0.95]), 2)
    stats_all_pars[par].columns = stat_list
    stats_all_pars[par]['IQR'] = stats_all_pars[par]['75th pc'] - stats_all_pars[par]['25th pc']

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_all_par_lc_on.xlsx'))
for par in sorted(slow_measurements_all.keys())[:-1]:
    stats_all_pars[par].loc[leak_comp_on].to_excel(writer, par)
writer.save()

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_all_par_lc_off.xlsx'))
for par in sorted(slow_measurements_all.keys())[:-1]:
    stats_all_pars[par].loc[leak_comp_off].to_excel(writer, par)
writer.save()

____

# 8. Compare leak-compensated and non-leak-compensated groups 

For **Leak%, leak_comp, MVleak and VT_diff** the mean and median are very different, these are not normally distributed. For these use non-parametric statistics (median, IQR). For the rest use parametric tests (mean, SD).

In [ ]:
# Histograms to visualize the mean values of the parameters
def leak_comp_hist(par, unit, stat = 'mean', bins = 10, lm = 10):
    '''
    Draws histogram for leak-comp on and leak-comp-off recordings
    '''
    fig = plt.figure(figsize = [6, 3])
    fig.add_subplot(1,2,1)
    plt.hist(stats_all_pars[par].loc[leak_comp_on][stat], 
             bins = bins, rwidth = 0.8, color = 'k')
    
    plt.ylim(0, lm)
    plt.title('Leak compensation ON', size = 12)
    plt.xlabel('%s %s (%s)' % (stat, par, unit))
    plt.ylabel('number of recordings')
    fig.add_subplot(1,2,2)
    plt.hist(stats_all_pars[par].loc[leak_comp_off][stat], 
             bins = bins, rwidth = 0.8, color = 'k')
    plt.xlabel('%s %s (%s)' % (stat, par, unit))
    plt.ylim(0, lm)
    plt.title('Leak compensation OFF', size = 12)

#### What is the distribution of the recording means of the various parameters

In [ ]:
leak_comp_hist('VTmand_kg', 'mL/kg')

In [ ]:
leak_comp_hist('leak%', '', 'median', lm = 15)

As the graphs show the means are not normally distributed, **group medians** need to be compared using **non-parametric tests**

In [ ]:
leak_comp_hist('VT_diff', 'mL/kg', 'median', lm = 20)

### Compare the mean values of the parameters except *leak%, leak_kg, leak_kg_2, leak_comp, MVleak_kg and VT_diff* for which compare the medians. Use non-parametric statistics for groups: group medians

##### Create a DataFrame for the mean values of the parametrically distributed parameters in each recordings

In [ ]:
select_pars_3 = ['VTimand_kg',  'VTmand_kg', 'VTemand_kg', 'VTset_kg',
                 'RR', 'RR_set', 'RRmand', 'MV_kg', 'MVemand_kg', 
                 'PIP', 'Pmax', 'Pmean', 'PEEP', 'P_diff', 'FiO2']

In [ ]:
means = {}

for par in select_pars_3:
    means[par] = stats_all_pars[par]['mean']

stats_all_mean = DataFrame(means)

In [ ]:
stats_all_mean.loc[leak_comp_on];

In [ ]:
stats_all_mean.loc[leak_comp_off];

##### Group statistics

In [ ]:
group_stats_on = stats_all_mean.loc[leak_comp_on].describe().T
group_stats_on = group_stats_on[['50%', '25%', '75%', 'min', 'max']]
group_stats_on.columns = ['median', '25%', '75%', 'min', 'max']

In [ ]:
group_stats_off = stats_all_mean.loc[leak_comp_off].describe().T
group_stats_off = group_stats_off[['50%', '25%', '75%', 'min', 'max']]
group_stats_off.columns = ['median', '25%', '75%', 'min', 'max']

In [ ]:
group_stats = pd.concat([group_stats_on, group_stats_off], axis = 1, 
                        keys = ['leak compensation ON', 'leak compensation OFF'])

In [ ]:
mann_whitney_p = []
for par in group_stats.index:
     mann_whitney_p.append(round(stats.mannwhitneyu(stats_all_mean.loc[leak_comp_on][par], 
                                                 stats_all_mean.loc[leak_comp_off][par])[1], 3))
group_stats['p value'] = mann_whitney_p

In [ ]:
group_stats

In [ ]:
select_pars_4 = ['MV_kg', 'VTmand_kg', 'P_diff']

In [ ]:
group_stats_sel = group_stats.loc[select_pars_4]
group_stats_sel

##### Create a DataFrame for the median values of the parametrically distributed parameters in each recordings

In [ ]:
select_pars_5 = ['leak%', 'leak_comp', 'MVleak_kg', 'VT_diff']

In [ ]:
medians = {}

for par in select_pars_5:
    medians[par] = stats_all_pars[par]['median']

stats_all_median = DataFrame(medians)

In [ ]:
stats_all_median.loc[leak_comp_on];

In [ ]:
stats_all_median.loc[leak_comp_off];

##### Group statistics

In [ ]:
group_stats_on_2 = stats_all_median.loc[leak_comp_on].describe().T
group_stats_on_2 = group_stats_on_2[['50%', '25%', '50%', 'min', 'max']]
group_stats_on_2.columns = ['median', '25%', '75%', 'min', 'max']

In [ ]:
group_stats_off_2 = stats_all_median.loc[leak_comp_off].describe().T
group_stats_off_2 = group_stats_off_2[['50%', '25%', '75%','min', 'max']]
group_stats_off_2.columns = ['median', '25%', '75%', 'min', 'max']

In [ ]:
group_stats_2 = pd.concat([group_stats_on_2, group_stats_off_2], axis = 1, 
                        keys = ['leak compensation ON', 'leak compensation OFF'])

In [ ]:
mann_whitney_p = []
for par in group_stats_2.index:
     mann_whitney_p.append(round(stats.mannwhitneyu(stats_all_median.loc[leak_comp_on][par], 
                                                 stats_all_median.loc[leak_comp_off][par])[1], 3))
group_stats_2['p value'] = mann_whitney_p

In [ ]:
group_stats_2

In [ ]:
select_pars_6 = ['leak%', 'VT_diff']

In [ ]:
group_stats_2_sel = group_stats_2.loc[select_pars_6]
group_stats_2_sel

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_means.xlsx'))
stats_all_mean.to_excel(writer, 'mean_all')
stats_all_mean.loc[leak_comp_on].to_excel(writer, 'mean_lc_on')
stats_all_mean.loc[leak_comp_off].to_excel(writer, 'mean_lc_off')
group_stats.to_excel(writer, 'mean_group_stats')
group_stats_sel.to_excel(writer, 'mean_group_stats_sel')
stats_all_median.to_excel(writer, 'median_all')
stats_all_median.loc[leak_comp_on].to_excel(writer, 'median_lc_on')
stats_all_median.loc[leak_comp_off].to_excel(writer, 'median_lc_off')
group_stats_2.to_excel(writer, 'median_group_stats')
group_stats_2_sel.to_excel(writer, 'median_group_stats_sel')
writer.save()

_____

# 9. Compare leak-compensated and non-leak-compensated groups but only consider only inflations with leak >50%

### Calculate descriptive statistics in each recording for all the parameters 

In [ ]:
slow_measurements_all_over50 = slow_measurements_all[slow_measurements_all['leak%'] > 50]

In [ ]:
len(slow_measurements_all_over50)

In [ ]:
len(slow_measurements_all_over50) / len(slow_measurements_all)

In [ ]:
len(slow_measurements_all_over50[slow_measurements_all_over50['recording'].isin(leak_comp_on)])

In [ ]:
len(slow_measurements_all_over50[slow_measurements_all_over50['recording'].isin(leak_comp_on)]) /\
    len(slow_measurements_all[slow_measurements_all['recording'].isin(leak_comp_on)])


In [ ]:
len(slow_measurements_all_over50[slow_measurements_all_over50['recording'].isin(leak_comp_off)])

In [ ]:
len(slow_measurements_all_over50[slow_measurements_all_over50['recording'].isin(leak_comp_off)]) /\
    len(slow_measurements_all[slow_measurements_all['recording'].isin(leak_comp_off)])

In [ ]:
stats_all_recs_over50 = {}

for recording in recordings:
    
    stats_all_recs_over50[recording] = \
        round(slow_measurements_all_over50[slow_measurements_all_over50['recording'] == 
        recording].describe(percentiles = [0.05, 0.25, 0.5, 0.75, 0.95]), 2)
    stats_all_recs_over50[recording].index = stat_list
    stats_all_recs_over50[recording] = stats_all_recs_over50[recording].T
    stats_all_recs_over50[recording]['IQR'] = stats_all_recs_over50[recording]['75th pc'] - \
                                                    stats_all_recs_over50[recording]['25th pc']

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_all_rec_lc_on.xlsx'))
for rec in leak_comp_on:
    stats_all_recs_over50[rec].to_excel(writer, rec)
writer.save()

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_all_rec_lc_off.xlsx'))
for rec in leak_comp_off:
    stats_all_recs_over50[rec].to_excel(writer, rec)
writer.save()

### Calculate descriptive statistics in all recordings for each of the parameters 

In [ ]:
recs = slow_measurements_all_over50.groupby('recording')

In [ ]:
stats_all_pars_over50 = {}

for par in sorted(slow_measurements_all_over50.keys())[:-1]: # removes the 'recording' field
    stats_all_pars_over50[par] = \
        round(recs[par].describe(percentiles = [0.05, 0.25, 0.5, 0.75, 0.95]), 2)
    stats_all_pars_over50[par].columns = stat_list
    stats_all_pars_over50[par]['IQR'] = stats_all_pars_over50[par]['75th pc'] \
                - stats_all_pars_over50[par]['25th pc']

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_all_par_lc_on_over50.xlsx'))
for par in sorted(slow_measurements_all.keys())[:-1]:
    stats_all_pars_over50[par].loc[leak_comp_on].to_excel(writer, par)
writer.save()

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_all_par_lc_off_over50.xlsx'))
for par in sorted(slow_measurements_all.keys())[:-1]:
    stats_all_pars_over50[par].loc[leak_comp_off].to_excel(writer, par)
writer.save()

### Compare leak-compensated and non-leak-compensated groups

For **Leak%, leak_comp, MVleak and VT_diff** the mean and median are very different, these are not normally distributed. For these use non-parametric statistics (median, IQR). For the rest use parametric tests (mean, SD).

In [ ]:
# Histograms to visualize the mean values of the parameters
def leak_comp_hist_over50(par, unit, stat = 'mean', bins = 10, lm = 10):
    '''
    Draws histogram for leak-comp on and leak-comp-off recordings
    '''
    fig = plt.figure(figsize = [6, 3])
    fig.add_subplot(1,2,1)
    plt.hist(stats_all_pars_over50[par].loc[leak_comp_on][stat], 
             bins = bins, rwidth = 0.8, color = 'k')
    
    plt.ylim(0, lm)
    plt.title('Leak compensation ON', size = 12)
    plt.xlabel('%s %s (%s)' % (stat, par, unit))
    plt.ylabel('number of recordings')
    fig.add_subplot(1,2,2)
    plt.hist(stats_all_pars_over50[par].loc[leak_comp_off][stat], 
             bins = bins, rwidth = 0.8, color = 'k')
    plt.xlabel('%s %s (%s)' % (stat, par, unit))
    plt.ylim(0, lm)
    plt.title('Leak compensation OFF', size = 12)

#### What is the distribution of the recording means of the various parameters

In [ ]:
leak_comp_hist_over50('VTmand_kg', 'mL/kg')

In [ ]:
leak_comp_hist_over50('leak%', 'median', lm = 15)

As the graphs show the means are not normally distributed, **group medians** need to be compared using **non-parametric tests**

In [ ]:
leak_comp_hist_over50('VT_diff', 'mL/kg', 'median', lm = 20)

### Compare the mean values of the parameters except *leak%,  leak_comp, MVleak_kg and VT_diff* for which compare the medians. Use non-parametric statistics for groups: group medians

##### Create a DataFrame for the mean values of the parametrically distributed parameters in each recordings

In [ ]:
select_pars_3 = ['VTimand_kg',  'VTmand_kg', 'VTemand_kg', 'VTset_kg',
                 'RR', 'RR_set', 'RRmand', 'MV_kg', 'MVemand_kg', 
                 'PIP', 'Pmax', 'Pmean', 'PEEP', 'P_diff', 'FiO2']

In [ ]:
means = {}

for par in select_pars_3:
    means[par] = stats_all_pars_over50[par]['mean']

stats_all_mean_over50 = DataFrame(means)

In [ ]:
stats_all_mean_over50.loc[leak_comp_on];

In [ ]:
stats_all_mean_over50.loc[leak_comp_off];

##### Group statistics

In [ ]:
group_stats_on = stats_all_mean_over50.loc[leak_comp_on].describe().T
group_stats_on = group_stats_on[['50%', '25%', '75%', 'min', 'max']]
group_stats_on.columns = ['median', '25%', '75%', 'min', 'max']

In [ ]:
group_stats_off = stats_all_mean_over50.loc[leak_comp_off].describe().T
group_stats_off = group_stats_off[['50%', '25%', '75%', 'min', 'max']]
group_stats_off.columns = ['median', '25%', '75%', 'min', 'max']

In [ ]:
group_stats = pd.concat([group_stats_on, group_stats_off], axis = 1, 
                        keys = ['leak compensation ON', 'leak compensation OFF'])

In [ ]:
mann_whitney_p = []
for par in group_stats.index:
     mann_whitney_p.append(round(stats.mannwhitneyu(stats_all_mean_over50.loc[leak_comp_on][par], 
                                                 stats_all_mean_over50.loc[leak_comp_off][par])[1], 3))
group_stats['p value'] = mann_whitney_p

In [ ]:
group_stats

In [ ]:
select_pars_4 = ['MV_kg', 'VTmand_kg', 'P_diff']

In [ ]:
group_stats_sel = group_stats.loc[select_pars_4]
group_stats_sel

##### Create a DataFrame for the median values of the parametrically distributed parameters in each recordings

In [ ]:
select_pars_5 = ['leak%', 'leak_comp', 'MVleak_kg', 'VT_diff']

In [ ]:
medians = {}

for par in select_pars_5:
    medians[par] = stats_all_pars_over50[par]['median']

stats_all_median_over50 = DataFrame(medians)

In [ ]:
stats_all_median_over50.loc[leak_comp_on];

In [ ]:
stats_all_median_over50.loc[leak_comp_off];

##### Group statistics

In [ ]:
group_stats_on_2 = stats_all_median_over50.loc[leak_comp_on].describe().T
group_stats_on_2 = group_stats_on_2[['50%', '25%', '75%', 'min', 'max']]
group_stats_on_2.columns = ['median', '25%', '75%', 'min', 'max']

In [ ]:
group_stats_off_2 = stats_all_median_over50.loc[leak_comp_off].describe().T
group_stats_off_2 = group_stats_off_2[['50%', '25%', '75%', 'min', 'max']]
group_stats_off_2.columns = ['median', '25%', '75%', 'min', 'max']

In [ ]:
group_stats_2 = pd.concat([group_stats_on_2, group_stats_off_2], axis = 1, 
                        keys = ['leak compensation ON', 'leak compensation OFF'])

In [ ]:
mann_whitney_p = []
for par in group_stats_2.index:
     mann_whitney_p.append(round(stats.mannwhitneyu(stats_all_median_over50.loc[leak_comp_on][par], 
                                                 stats_all_median_over50.loc[leak_comp_off][par])[1], 3))
group_stats_2['p value'] = mann_whitney_p

In [ ]:
group_stats_2

In [ ]:
select_pars_6 = ['leak%', 'VT_diff']

In [ ]:
group_stats_2_sel = group_stats_2.loc[select_pars_6]
group_stats_2_sel

In [ ]:
writer = pd.ExcelWriter('%s/%s' % (DIR_WRITE, 'stats_means_over50.xlsx'))
stats_all_mean_over50.to_excel(writer, 'mean_all')
stats_all_mean_over50.loc[leak_comp_on].to_excel(writer, 'mean_lc_on')
stats_all_mean_over50.loc[leak_comp_off].to_excel(writer, 'mean_lc_off')
group_stats.to_excel(writer, 'mean_group_stats')
group_stats_sel.to_excel(writer, 'mean_group_stats_sel')
stats_all_median_over50.to_excel(writer, 'median_all')
stats_all_median_over50.loc[leak_comp_on].to_excel(writer, 'median_lc_on')
stats_all_median_over50.loc[leak_comp_off].to_excel(writer, 'median_lc_off')
group_stats_2.to_excel(writer, 'median_group_stats')
group_stats_2_sel.to_excel(writer, 'median_group_stats_sel')
writer.save()

____

# 10. Compare pCO2 levels in the recordings with or without leak compensation

#### Import blood gases and preprocess pCO2 data

In [ ]:
blood_gases = {}
for recording in recordings:
    blood_gas_loader('%s/%s' % (CWD, 'data_grabber_gases_old.xlsx'), blood_gases, recording)                   

In [ ]:
pCO2s = {}
for recording in recordings:
    pCO2s[recording] = blood_gases[recording][['pCO2, POC', 'Blood specimen type, POC']]

In [ ]:
# Change the index of pCO2s into single index format

for recording in recordings:
    # print('processing CO2 data for %s' % recording)
    time_list_all = []
    for i in range(len(pCO2s[recording])):
        day = str(pCO2s[recording].index[i][0])[:10]
        time = str(pCO2s[recording].index[i][1])
        date_time = day + ' ' + time
        time_list_all.append(date_time)
                   
    pCO2s[recording].index = time_list_all

In [ ]:
# Convert the indices of the pCO2s DataFrames to datetime index

for recording in recordings:
    pCO2s[recording].index = pd.to_datetime(pCO2s[recording].index)

In [ ]:
# Limit pCO2 data to the time of the recordings

dt_format = '%Y-%m-%d %H:%M:%S'

pCO2s_filtered = {}

for recording in recordings:
    a = slow_measurements_all[slow_measurements_all['recording'] == recording]
    
    pCO2s_filtered[recording] = \
        pCO2s[recording][a.index[0].strftime(dt_format) :
                         a.index[-1].strftime(dt_format)]   

In [ ]:
sum(pCO2s_filtered[recording]['Blood specimen type, POC'] == 'Venous')

In [ ]:
# How many blood gases did the different recordings have?

gas_stats = {}

for recording in recordings:
    gas_stats[recording] = [len(pCO2s_filtered[recording]),
                            sum(pCO2s_filtered[recording]['Blood specimen type, POC'] == 'Arteri...'),
                            sum(pCO2s_filtered[recording]['Blood specimen type, POC'] == 'Capill...'),
                            sum(pCO2s_filtered[recording]['Blood specimen type, POC'] == 'Venous...')]
gas_stats = DataFrame(gas_stats).T
gas_stats.columns = ['total', 'arterial', 'capillary', 'venous']
gas_stats.index.name = 'recording'

In [ ]:
gas_stats;

In [ ]:
gas_stats.sum()

In [ ]:
gas_stats.loc[leak_comp_on].sum()

In [ ]:
gas_stats.loc[leak_comp_off].sum()

#### Consider all kind of blood gases

In [ ]:
CO2_lc_on = {}
CO2_lc_off = {}

for key in sorted(pCO2s_filtered.keys()):
    if key in leak_comp_on:
        CO2_lc_on[key] = pCO2s_filtered[key]
    elif key in leak_comp_off:
        CO2_lc_off[key] = pCO2s_filtered[key]

In [ ]:
CO2_lc_on = pd.concat(CO2_lc_on)
CO2_lc_off = pd.concat(CO2_lc_off)

In [ ]:
len(CO2_lc_on), len(CO2_lc_off)

In [ ]:
plt.figure(figsize = (4,4))
plt.hist(CO2_lc_on['pCO2, POC'], bins = 20, color = 'black');

In [ ]:
plt.figure(figsize = (4,4))
plt.hist(CO2_lc_off['pCO2, POC'], bins = 20, color = 'black');

In [ ]:
CO2_lc_on['pCO2, POC'].median()

In [ ]:
np.percentile(CO2_lc_on['pCO2, POC'], 25), np.percentile(CO2_lc_on['pCO2, POC'], 75) 

In [ ]:
CO2_lc_off['pCO2, POC'].median()

In [ ]:
np.percentile(CO2_lc_off['pCO2, POC'], 25), np.percentile(CO2_lc_off['pCO2, POC'], 75) 

In [ ]:
stats.mannwhitneyu(CO2_lc_on['pCO2, POC'].dropna(), CO2_lc_off['pCO2, POC'].dropna())

#### Consider arterial gases only

In [ ]:
pCO2s_filtered_arterial = {}

for recording in pCO2s_filtered.keys():
    a = pCO2s_filtered[recording]
    pCO2s_filtered_arterial[recording] = a[a['Blood specimen type, POC'] == 'Arteri...']
    

In [ ]:
CO2_lc_on_art = {}
CO2_lc_off_art = {}

for key in sorted(pCO2s_filtered_arterial.keys()):
    if key in leak_comp_on:
        CO2_lc_on_art[key] = pCO2s_filtered_arterial[key]
    elif key in leak_comp_off:
        CO2_lc_off_art[key] = pCO2s_filtered_arterial[key]

In [ ]:
CO2_lc_on_art = pd.concat(CO2_lc_on_art)
CO2_lc_off_art = pd.concat(CO2_lc_off_art)

In [ ]:
len(CO2_lc_on_art), len(CO2_lc_off_art)

In [ ]:
plt.figure(figsize = (4,4))
plt.hist(CO2_lc_on_art['pCO2, POC'], bins = 20, color = 'black');

In [ ]:
plt.figure(figsize = (4,4))
plt.hist(CO2_lc_off_art['pCO2, POC'], bins = 20, color = 'black');

In [ ]:
round(CO2_lc_on_art['pCO2, POC'].median(), 2)

In [ ]:
np.percentile(CO2_lc_on_art['pCO2, POC'], 25), np.percentile(CO2_lc_on_art['pCO2, POC'], 75) 

In [ ]:
round(CO2_lc_off_art['pCO2, POC'].median(), 2)

In [ ]:
np.percentile(CO2_lc_off_art['pCO2, POC'], 25), np.percentile(CO2_lc_off_art['pCO2, POC'], 75) 

In [ ]:
stats.mannwhitneyu(CO2_lc_on_art['pCO2, POC'].dropna(), CO2_lc_off_art['pCO2, POC'].dropna())

_____

# 11. What was the number of relevant alarms during the leak compensation on and leak compensation off recordings

### Import alarm states

In [ ]:
alarm_states = {}

for recording in recordings:
    flist = os.listdir('%s/%s' % (DIR_READ_2, recording))
    flist = [file for file in flist if not file.startswith('.')] # There are some hidden 
    # files on the hard drive starting with '.'; this step is necessary to ignore them
    files = alarm_state_finder(flist)
    # print('Loading recording %s' % recording)
    # print(files)
    fnames = ['%s/%s/%s' % (DIR_READ_2, recording, filename) for filename in files]
    alarm_states[recording] =  data_loader(fnames)

In [ ]:
# Limit alarm events for the duration of the recordings

for recording in recordings:
    
    if slow_measurements[recording].index[0] <= alarm_states[recording].index[0]:
        continue
    else:
        alarm_states[recording] = alarm_states[recording][slow_measurements[recording].index[0] : ]
        
    if slow_measurements[recording].index[-1] >= alarm_states[recording].index[-1]:
        continue
    else:
        alarm_states[recording] = alarm_states[recording][ : slow_measurements[recording].index[-1]]

In [ ]:
# Whics alarm occur in the recordings?

all_alarms = set()
for recording in recordings:
    all_alarms.update(alarm_states[recording]['Name'].unique())  

all_alarms

The relevant alarm categories are:

*  'Tidal volume < low Limit'
*  'Tube obstructed'
*  'Volume not constant'

In [ ]:
alarm_count = {}
alarm_count_norm = {} # alarm count is normalized to the length of the recording (in hours)

for recording in recordings:
    a = alarm_states[recording]
    b = a[a['Name'].isin(['Tidal volume < low Limit', 'Tube obstructed','Volume not constant'])]
    c = b[b['State New'] == 'Active']
    
    alarm_count[recording] = len(c) 
    alarm_count_norm[recording] = len(c) / (len(slow_measurements[recording]) / 3600)

alarm_count = DataFrame([alarm_count]).T
alarm_count.index.name = 'recording'
alarm_count.columns = ['alarm events']

alarm_count_norm = DataFrame([alarm_count_norm]).T
alarm_count_norm.index.name = 'recording'
alarm_count_norm.columns = ['alarm events/hour of recording']

#### Look at the total alarm counts in the leak comp and non-leak comp recordings normalized for the total lengths of the recordings for each group

In [ ]:
alarm_count.loc[leak_comp_on]

In [ ]:
alarm_count.loc[leak_comp_off]

In [ ]:
alarm_count_all_lc = alarm_count.loc[leak_comp_on].sum() / (len(slow_measurements_all_lc) / 3600)
alarm_count_all_no_lc = alarm_count.loc[leak_comp_off].sum() / (len(slow_measurements_all_no_lc) / 3600)

In [ ]:
alarm_count_all_lc, alarm_count_all_no_lc

#### Look at the number of alarms / hour in the individual recordings

In [ ]:
alarm_count_norm.loc[leak_comp_on]

In [ ]:
alarm_count_norm.loc[leak_comp_off]

In [ ]:
alarm_count_norm.loc[leak_comp_on].median()

In [ ]:
alarm_count_norm.loc[leak_comp_on].mean()

In [ ]:
alarm_count_norm.loc[leak_comp_off].median()

In [ ]:
alarm_count_norm.loc[leak_comp_off].mean()

In [ ]:
# alarm count / hour

fig, axes = plt.subplots(1,2, sharey = False, figsize = (7,3))
axes[0].hist(alarm_count_norm.loc[leak_comp_on].values, color = 'black', alpha = 0.75);
axes[1].hist(alarm_count_norm.loc[leak_comp_off].values, color = 'black', alpha = 0.75);
axes[0].set_ylabel('number of recordings'); axes[1].set_ylabel('number of recordings')
axes[0].set_title('Leak compensation ON', size = 12); axes[1].set_title('Leak compensation OFF', size = 12)
axes[0].set_xlabel('number of alarms per hour'); axes[1].set_xlabel('number of alarms per hour')
axes[1].yaxis.tick_right(); axes[1].yaxis.set_label_position("right")

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'alarm_count', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
stats.mannwhitneyu(alarm_count_norm.loc[leak_comp_on], alarm_count_norm.loc[leak_comp_off])

In [ ]:
stats.ttest_ind(alarm_count_norm.loc[leak_comp_on], alarm_count_norm.loc[leak_comp_off])

In [ ]:
(alarm_count_norm.loc[leak_comp_on], alarm_count_norm.loc[leak_comp_off])

_____

# 12. Figures for the paper

### Figure 1

In [ ]:
fig = plt.figure()
fig.set_size_inches(8, 4)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
ax = fig.add_subplot(1, 1, 1)
rec = 'DG013'
vent0 = slow_measurements_all_no_lc[slow_measurements_all_no_lc['recording'] == rec].resample('1min').mean()
vent0 = vent0.loc['2015-11-25 21:00:00' : '2015-11-26 03:00:00' ]
vent1 = vent0['VTimand_kg']
vent3 = vent0['VTemand_kg']

vent1.plot(ax = ax, color = 'red', ylim = [0, 20])
vent3.plot(ax = ax, color = 'green', ylim = [0, 20])
vent0['VTset_kg'].plot(ax = ax, color = 'black', linewidth = 1, linestyle = 'dashed')

ax.set_ylim(0, 12)
ax.set_xlabel('Time (hours)', size = 12, color = 'black')
ax.set_ylabel('mL/kg', size = 12, color = 'black')
ax.set_title('Tidal volumes',  size = 12, color = 'black')
ax.legend(['VTimand', 'VTemand', 'VT_set'], fontsize = 12)
ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax.tick_params(which = 'both', labelsize=12)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1A', 'jpg'),
            dpi = 300, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
fig = plt.figure()
fig.set_size_inches(8, 4)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
ax = fig.add_subplot(1, 1, 1)
rec = 'DG032_2'
vent0 = slow_measurements_all_lc[slow_measurements_all_lc['recording'] == rec].resample('1min').mean()
vent0 = vent0.loc['2016-03-25 01:00:00' : '2016-03-25 07:00:00' ]
vent1 = vent0['VTimand_kg']
vent2 = vent0['VTmand_kg']
vent3 = vent0['VTemand_kg']

vent1.plot(ax = ax, color = 'red', ylim = [0, 20])
vent2.plot(ax = ax, color = 'blue', ylim = [0, 20])
vent3.plot(ax = ax, color = 'green', ylim = [0, 20])
vent0['VTset_kg'].plot(ax = ax, color = 'black', linewidth = 1, linestyle = 'dashed')

ax.axvline(pd.to_datetime('2016-03-25 04:10:00'), color='black', linestyle='dotted', lw=1)
ax.axvline(pd.to_datetime('2016-03-25 04:40:00'), color='black', linestyle='dotted', lw=1)

ax.set_ylim(3, 12)
ax.set_xlabel('Time (hours)', size = 12, color = 'black')
ax.set_ylabel('mL/kg', size = 12, color = 'black')
ax.set_title('Tidal volumes',  size = 12, color = 'black')
ax.legend(['VTimand', 'VTmand', 'VTemand', 'VT_set'], fontsize = 12)
ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax.tick_params(which = 'both', labelsize=12)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1B', 'jpg'),
            dpi = 300, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
fig = plt.figure()
fig.set_size_inches(8, 4)
fig.subplots_adjust(left=None, bottom=0.2, right=None, top=None, wspace=None, hspace=None)
ax = fig.add_subplot(1, 1, 1)
rec = 'DG032_2'
vent0 = slow_measurements_all_lc[slow_measurements_all_lc['recording'] == rec].resample('1min').mean()
vent0 = vent0.loc['2016-03-25 04:10:00' : '2016-03-25 04:40:00' ]
vent1 = vent0['VTimand_kg']
vent2 = vent0['VTmand_kg']
vent3 = vent0['VTemand_kg']

vent1.plot(ax = ax, color = 'red', ylim = [0, 20])
vent2.plot(ax = ax, color = 'blue', ylim = [0, 20])
vent3.plot(ax = ax, color = 'green', ylim = [0, 20])

vent0['VTset_kg'].plot(ax = ax, color = 'black', linewidth = 1, linestyle = 'dashed')
ax.set_ylim(3, 12)
ax.set_xlabel('Time (hours)', size = 12, color = 'black')
ax.set_ylabel('mL/kg', size = 12, color = 'black')
ax.set_title('Tidal volumes',  size = 12, color = 'black')
ax.legend(['VTimand', 'VTmand', 'VTemand', 'VT_set'], fontsize = 12)
ax.grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax.tick_params(which = 'both', labelsize=12)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1C', 'jpg'),
            dpi = 300, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

### Figure 2

In [ ]:
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']

fig, ax = plt.subplots(figsize = [8,6])
cats_leak_pc_lc.size().plot(kind = 'bar', logy = False, 
                        title = 'Leak-compensated inflations', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xticklabels(labels, rotation = 0)
ax.set_xlabel('leak (%)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2A', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
bins = [0,1,2,21]

cats_leak_comp_custom = pd.cut(slow_measurements_all_lc['leak_comp'], 
                              bins = bins, right = False)
cats_leak_comp_custom.value_counts().sort_index()

In [ ]:
(cats_leak_comp_custom.value_counts() / len(cats_leak_comp)) * 100

In [ ]:
labels = ['0-1', '1-2', '2-20']

fig, ax = plt.subplots(figsize = [4,6])
cats_leak_comp_custom.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'Leak-compensated inflations', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xticklabels(labels, rotation = 0)
ax.set_xlabel('Leak compensation (mL/kg)', size = 14)
ax.set_ylabel('number of inflations', size = 14)
ax.set_ylim(0,3600000)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2B', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
bins = [0, 0.2, 0.4, 0.6 , 0.8, 1.001]

cats_leak_comp_zoom2 = pd.cut(slow_measurements_all_lc['leak_comp'], 
                              bins = bins, right = False)
cats_leak_comp_zoom2.value_counts().sort_index()

In [ ]:
labels = ['0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1']

fig, ax = plt.subplots(figsize = [6,6])
cats_leak_comp_zoom2.value_counts().sort_index().plot(kind = 'bar', logy = False, 
                        title = 'leak compensation', color = 'black', alpha = 0.7, fontsize = 14)
ax.set_xlabel('range (mL/kg)', size = 14)
ax.set_ylabel('number of breaths', size = 14)
ax.set_xticklabels(labels, rotation = 0)
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2C', 'jpg'),
    dpi = 200, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = 'jpg',
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(figsize = (8,6))
data = []
for name, group in grouped_lc:
    data.append(group['leak_comp'].values)
plt.boxplot(data, whis = [5, 95], showfliers = False,showmeans = True,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('Leak compensation (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2D', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

### Figure 3

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(figsize = (8,6))
data = []
for name, group in grouped_no_lc:
    data.append(group['VT_diff'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False, showmeans = True,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('VTdiff (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_3A', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(figsize = (8,6))
data = []
for name, group in grouped_lc:
    data.append(group['VT_diff'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False, showmeans = True,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('VTdiff (mL/kg)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_3B', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(figsize = (8,6))
data = []
for name, group in grouped_no_lc:
    data.append(group['P_diff'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False, showmeans = True,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('Pdiff (mbar)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_3C', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
dpi = 300
filetype = 'jpg'
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}

fig, ax = plt.subplots(figsize = (8,6))
data = []
for name, group in grouped_lc:
    data.append(group['P_diff'].dropna().values)
plt.boxplot(data, whis = [5, 95], showfliers = False, showmeans = True,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

plt.xticks(np.arange(1, len(data)+1), xticklabels, rotation = 'horizontal')
plt.xlabel('Leak (%)', size = 14)
plt.ylabel('Pdiff (mbar)', size = 14)
ax.tick_params(axis='both', which='major', labelsize=14)

plt.grid('on')
plt.tight_layout()

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_3D', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
    transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

## Combined Figures for paper

In [ ]:
dpi = 400
filetype = 'tiff'

fig, ax = plt.subplots(3, 1, figsize = (6, 8), dpi = 400)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0.3, wspace=0.3)

rec = 'DG013'
vent0 = slow_measurements_all_no_lc[slow_measurements_all_no_lc['recording'] == rec].resample('1min').mean()
vent0 = vent0.loc['2015-11-25 21:00:00' : '2015-11-26 03:00:00' ]
vent1 = vent0['VTimand_kg']
vent3 = vent0['VTemand_kg']

vent1.plot(ax = ax[0], color = 'black', alpha = 1, ylim = [0, 20])
vent3.plot(ax = ax[0], color = 'black', alpha = 0.3, ylim = [0, 20])
vent0['VTset_kg'].plot(ax = ax[0], color = 'black', linewidth = 0.75, linestyle = 'dashed', dashes = [2,2])

ax[0].set_ylim(0, 16)
ax[0].set_xlabel('', size = 10, color = 'black')
ax[0].set_ylabel('mL/kg', size = 10, color = 'black')
ax[0].set_title('Tidal volumes',  size = 10, color = 'black')
ax[0].legend(['VTi', 'VTe', 'VTset'], fontsize = 8)
ax[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax[0].tick_params(which = 'both', labelsize=10)


rec = 'DG032_2'
vent0 = slow_measurements_all_lc[slow_measurements_all_lc['recording'] == rec].resample('1min').mean()
vent0 = vent0.loc['2016-03-25 01:00:00' : '2016-03-25 07:00:00' ]
vent1 = vent0['VTimand_kg']
vent2 = vent0['VTmand_kg']
vent3 = vent0['VTemand_kg']

vent1.plot(ax = ax[1], color = 'black', alpha = 1, ylim = [0, 20])
vent2.plot(ax = ax[1], color = 'black', alpha = 0.5, ylim = [0, 20])
vent3.plot(ax = ax[1], color = 'black', alpha = 0.3, ylim = [0, 20])
vent0['VTset_kg'].plot(ax = ax[1], color = 'black', linewidth = 0.75, linestyle = 'dashed', dashes = [2,2])

ax[1].axvline(pd.to_datetime('2016-03-25 04:10:00'), color='black', linestyle='dotted', lw=1)
ax[1].axvline(pd.to_datetime('2016-03-25 04:40:00'), color='black', linestyle='dotted', lw=1)

ax[1].set_ylim(3, 14)
ax[1].set_xlabel('', size = 10, color = 'black')
ax[1].set_ylabel('mL/kg', size = 10, color = 'black')
ax[1].set_title('',  size = 10, color = 'black')
ax[1].legend(['VTi', 'VTlc', 'VT', 'VTset'], fontsize = 8)
ax[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax[1].tick_params(which = 'both', labelsize=10)


rec = 'DG032_2'
vent0 = slow_measurements_all_lc[slow_measurements_all_lc['recording'] == rec].resample('1min').mean()
vent0 = vent0.loc['2016-03-25 04:10:00' : '2016-03-25 04:40:00' ]
vent1 = vent0['VTimand_kg']
vent2 = vent0['VTmand_kg']
vent3 = vent0['VTemand_kg']

vent1.plot(ax = ax[2], color = 'black', alpha = 1, ylim = [0, 20])
vent2.plot(ax = ax[2], color = 'black', alpha = 0.5, ylim = [0, 20])
vent3.plot(ax = ax[2], color = 'black', alpha = 0.3, ylim = [0, 20])

vent0['VTset_kg'].plot(ax = ax[2], color = 'black', linewidth = 0.75, linestyle = 'dashed', dashes = [2,2])
ax[2].set_ylim(3, 12)
ax[2].set_xlabel('Time (hours)', size = 10, color = 'black')
ax[2].set_ylabel('mL/kg', size = 10, color = 'black')
ax[2].set_title('',  size = 10, color = 'black')
ax[2].legend(['VTi', 'VTlc', 'VTe', 'VTset'], fontsize = 8)
ax[2].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax[2].tick_params(which = 'both', labelsize=10)

fig.text(0.04, 0.9, 'A', fontsize = 16); fig.text(0.04, 0.63, 'B', fontsize = 16)
fig.text(0.04, 0.33, 'C', fontsize = 16)

fig.text(0.15, 0.88, 'Leak compensation OFF', fontsize = 10)
fig.text(0.15, 0.59, 'Leak compensation ON', fontsize = 10)
fig.text(0.15, 0.30, 'Leak compensation ON', fontsize = 10)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1_bw', filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
dpi = 400
filetype = 'jpg'

fig, ax = plt.subplots(3, 1, figsize = (6, 8), dpi = 400)
fig.subplots_adjust(left=None, bottom=None, right=None, top=None, hspace=0.3, wspace=0.3)

rec = 'DG013'
vent0 = slow_measurements_all_no_lc[slow_measurements_all_no_lc['recording'] == rec].resample('1min').mean()
vent0 = vent0.loc['2015-11-25 21:00:00' : '2015-11-26 03:00:00' ]
vent1 = vent0['VTimand_kg']
vent3 = vent0['VTemand_kg']

vent1.plot(ax = ax[0], color = 'red', ylim = [0, 20])
vent3.plot(ax = ax[0], color = 'green', ylim = [0, 20])
vent0['VTset_kg'].plot(ax = ax[0], color = 'black', linewidth = 1, linestyle = 'dashed')

ax[0].set_ylim(0, 16)
ax[0].set_xlabel('', size = 10, color = 'black')
ax[0].set_ylabel('mL/kg', size = 10, color = 'black')
ax[0].set_title('Tidal volumes',  size = 10, color = 'black')
ax[0].legend(['VTi', 'VTe', 'VTset'], fontsize = 8)
ax[0].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax[0].tick_params(which = 'both', labelsize=10)


rec = 'DG032_2'
vent0 = slow_measurements_all_lc[slow_measurements_all_lc['recording'] == rec].resample('1min').mean()
vent0 = vent0.loc['2016-03-25 01:00:00' : '2016-03-25 07:00:00' ]
vent1 = vent0['VTimand_kg']
vent2 = vent0['VTmand_kg']
vent3 = vent0['VTemand_kg']

vent1.plot(ax = ax[1], color = 'red', ylim = [0, 20])
vent2.plot(ax = ax[1], color = 'blue', ylim = [0, 20])
vent3.plot(ax = ax[1], color = 'green', ylim = [0, 20])
vent0['VTset_kg'].plot(ax = ax[1], color = 'black', linewidth = 1, linestyle = 'dashed')

ax[1].axvline(pd.to_datetime('2016-03-25 04:10:00'), color='black', linestyle='dotted', lw=1)
ax[1].axvline(pd.to_datetime('2016-03-25 04:40:00'), color='black', linestyle='dotted', lw=1)

ax[1].set_ylim(3, 14)
ax[1].set_xlabel('', size = 10, color = 'black')
ax[1].set_ylabel('mL/kg', size = 10, color = 'black')
ax[1].set_title('',  size = 10, color = 'black')
ax[1].legend(['VTi', 'VTlc', 'VT', 'VTset'], fontsize = 8)
ax[1].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax[1].tick_params(which = 'both', labelsize=10)


rec = 'DG032_2'
vent0 = slow_measurements_all_lc[slow_measurements_all_lc['recording'] == rec].resample('1min').mean()
vent0 = vent0.loc['2016-03-25 04:10:00' : '2016-03-25 04:40:00' ]
vent1 = vent0['VTimand_kg']
vent2 = vent0['VTmand_kg']
vent3 = vent0['VTemand_kg']

vent1.plot(ax = ax[2], color = 'red', ylim = [0, 20])
vent2.plot(ax = ax[2], color = 'blue', ylim = [0, 20])
vent3.plot(ax = ax[2], color = 'green', ylim = [0, 20])

vent0['VTset_kg'].plot(ax = ax[2], color = 'black', linewidth = 1, linestyle = 'dashed')
ax[2].set_ylim(3, 12)
ax[2].set_xlabel('Time (hours)', size = 10, color = 'black')
ax[2].set_ylabel('mL/kg', size = 10, color = 'black')
ax[2].set_title('',  size = 10, color = 'black')
ax[2].legend(['VTi', 'VTlc', 'VTe', 'VTset'], fontsize = 8)
ax[2].grid('on', linestyle='-', linewidth=0.5, color = 'gray')
ax[2].tick_params(which = 'both', labelsize=10)

fig.text(0.04, 0.9, 'A', fontsize = 16); fig.text(0.04, 0.63, 'B', fontsize = 16)
fig.text(0.04, 0.33, 'C', fontsize = 16)

fig.text(0.15, 0.88, 'Leak compensation OFF', fontsize = 10)
fig.text(0.15, 0.59, 'Leak compensation ON', fontsize = 10)
fig.text(0.15, 0.30, 'Leak compensation ON', fontsize = 10)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_1_color', filetype),
            dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
dpi = 400
filetype = 'jpg'

fig = plt.figure(figsize=(7, 7))
fig.subplots_adjust(left=0.2, bottom=0.1, right=0.9, top=0.9, 
                                        hspace=0.4, wspace=1)

ax0 = plt.subplot2grid((3, 8), (0, 0), colspan=8)
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']
cats_leak_pc_lc.size().plot(ax = ax0, kind = 'bar', logy = False, 
                        title = '', color = 'black', alpha = 0.7, fontsize = 10)
ax0.set_xticklabels(labels, rotation = 0)
ax0.set_xlabel('leak (%)', size = 10)
ax0.set_ylabel('number of inflations', size = 10)

ax1 = plt.subplot2grid((3, 8), (1, 0), colspan=3)
labels = ['0-1', '1-2', '2-20']
cats_leak_comp_custom.value_counts().sort_index().plot(ax = ax1, kind = 'bar', logy = False, 
                        title = '', color = 'black', alpha = 0.7, fontsize = 10)
ax1.set_xticklabels(labels, rotation = 0)
ax1.set_xlabel('Leak compensation (mL/kg)', size = 10)
ax1.set_ylabel('number of inflations', size = 10)
ax1.set_ylim(0,3600000)

ax2 = plt.subplot2grid((3, 8), (1, 3), colspan=6)
labels = ['0-0.2', '0.2-0.4', '0.4-0.6', '0.6-0.8', '0.8-1']
cats_leak_comp_zoom2.value_counts().sort_index().plot(ax = ax2, kind = 'bar', logy = False, 
                        title = '', color = 'black', alpha = 0.7, fontsize = 10)
ax2.set_xlabel('Leak compensation (mL/kg)', size = 10)
ax2.set_ylabel('', size = 10)
ax2.set_xticklabels(labels, rotation = 0)
ax2.set_ylim(0,3600000)
ax2.set_yticklabels('')


ax3 = plt.subplot2grid((3, 8), (2, 0), colspan=8)
xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='black', markersize = 3)

data = []
for name, group in grouped_lc:
    data.append(group['leak_comp'].values)
ax3.boxplot(data, whis = [5, 95], showfliers = False,showmeans = True, meanprops = meanprops,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)
ax3.set_xticks(np.arange(1, len(data)+1), xticklabels)
ax3.set_xticklabels(xticklabels, size = 10)
ax3.set_xlabel('Leak (%)', size = 10)
ax3.set_ylabel('Leak compensation (mL/kg)', size = 10)
ax3.grid('on')

fig.text(0.04, 0.9, 'A', fontsize = 16); fig.text(0.04, 0.63, 'B', fontsize = 16)
fig.text(0.04, 0.33, 'C', fontsize = 16)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_2', filetype),
    dpi = dpi, facecolor='w', edgecolor='w', orientation='portrait', papertype=None, 
    format = filetype, transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
dpi = 400
filetype = 'jpg'

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='black', markersize = 3)

fig, ax = plt.subplots(2,2, figsize = [12,6])
fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, 
                                        hspace=0.4, wspace=0.2)

xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']

# Figure 3A
data = []
for name, group in grouped_no_lc:
    data.append(group['VT_diff'].dropna().values)
ax[0,0].boxplot(data, whis = [5, 95], showfliers = False, showmeans = True, meanprops = meanprops, 
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)
ax[0,0].set_xticks(np.arange(1, len(data)+1), xticklabels)
ax[0,0].set_xticklabels(xticklabels, size = 10)
ax[0,0].set_xlabel('Leak (%)', size = 14)
ax[0,0].set_ylabel('VTdiff (mL/kg)', size = 14)
ax[0,0].grid('on')

# Figure 3B
data = []
for name, group in grouped_lc:
    data.append(group['VT_diff'].dropna().values)
ax[0,1].boxplot(data, whis = [5, 95], showfliers = False, showmeans = True, meanprops = meanprops, 
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

ax[0,1].set_xticks(np.arange(1, len(data)+1), xticklabels)
ax[0,1].set_xticklabels(xticklabels, size = 10)
ax[0,1].set_xlabel('Leak (%)', size = 14)
ax[0,1].set_ylabel('VTdiff (mL/kg)', size = 14)
ax[0,1].grid('on')

# Figure 3C
data = []
for name, group in grouped_no_lc:
    data.append(group['P_diff'].dropna().values)
ax[1,0].boxplot(data, whis = [5, 95], showfliers = False, showmeans = True, meanprops = meanprops,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)
ax[1,0].set_xticks(np.arange(1, len(data)+1), xticklabels)
ax[1,0].set_xticklabels(xticklabels, size = 10)
ax[1,0].set_xlabel('Leak (%)', size = 14)
ax[1,0].set_ylabel('Pdiff (mbar)', size = 14)
ax[1,0].grid('on')

# Figure 3D
data = []
for name, group in grouped_lc:
    data.append(group['P_diff'].dropna().values)
ax[1,1].boxplot(data, whis = [5, 95], showfliers = False, showmeans = True, meanprops = meanprops,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)
ax[1,1].set_xticks(np.arange(1, len(data)+1), xticklabels)
ax[1,1].set_xticklabels(xticklabels, size = 10)
ax[1,1].set_xlabel('Leak (%)', size = 14)
ax[1,1].set_ylabel('Pdiff (mbar)', size = 14)
ax[1,1].grid('on')


fig.text(0.04, 0.9, 'A', fontsize = 16); fig.text(0.5, 0.9, 'B', fontsize = 16)
fig.text(0.04, 0.5, 'C', fontsize = 16); fig.text(0.5, 0.5, 'D', fontsize = 16)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_3', filetype), dpi = dpi, 
            facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);

In [ ]:
dpi = 400
filetype = 'jpg'

# Define styling for each boxplot component
medianprops = {'color': 'black', 'linewidth': 2}
boxprops = {'color': 'black', 'linestyle': '-'}
whiskerprops = {'color': 'black', 'linestyle': '-'}
capprops = {'color': 'black', 'linestyle': '-'}
meanprops = dict(marker='D', markeredgecolor='black', markerfacecolor='black', markersize = 3)

fig, ax = plt.subplots(4,1, figsize = [6,10], dpi = 400)
fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9, 
                                        hspace=0.4, wspace=0.2)

xticklabels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79',
         '80-89', '90-99']

# Figure 3A
data = []
for name, group in grouped_no_lc:
    data.append(group['VT_diff'].dropna().values)
ax[0].boxplot(data, whis = [5, 95], showfliers = False, showmeans = True, meanprops = meanprops, 
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)
ax[0].set_xticks(np.arange(1, len(data)+1), xticklabels)
ax[0].set_xticklabels(xticklabels, size = 10)
ax[0].set_xlabel('Leak (%)', size = 14)
ax[0].set_ylabel('VTdiff (mL/kg)', size = 14)
ax[0].grid('on')

# Figure 3B
data = []
for name, group in grouped_lc:
    data.append(group['VT_diff'].dropna().values)
ax[1].boxplot(data, whis = [5, 95], showfliers = False, showmeans = True, meanprops = meanprops, 
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)

ax[1].set_xticks(np.arange(1, len(data)+1), xticklabels)
ax[1].set_xticklabels(xticklabels, size = 10)
ax[1].set_xlabel('Leak (%)', size = 14)
ax[1].set_ylabel('VTdiff (mL/kg)', size = 14)
ax[1].grid('on')

# Figure 3C
data = []
for name, group in grouped_no_lc:
    data.append(group['P_diff'].dropna().values)
ax[2].boxplot(data, whis = [5, 95], showfliers = False, showmeans = True, meanprops = meanprops,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)
ax[2].set_xticks(np.arange(1, len(data)+1), xticklabels)
ax[2].set_xticklabels(xticklabels, size = 10)
ax[2].set_xlabel('Leak (%)', size = 14)
ax[2].set_ylabel('Pdiff (mbar)', size = 14)
ax[2].grid('on')

# Figure 3D
data = []
for name, group in grouped_lc:
    data.append(group['P_diff'].dropna().values)
ax[3].boxplot(data, whis = [5, 95], showfliers = False, showmeans = True, meanprops = meanprops,
        medianprops=medianprops, boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops,)
ax[3].set_xticks(np.arange(1, len(data)+1), xticklabels)
ax[3].set_xticklabels(xticklabels, size = 10)
ax[3].set_xlabel('Leak (%)', size = 14)
ax[3].set_ylabel('Pdiff (mbar)', size = 14)
ax[3].grid('on')


fig.text(0.04, 0.92, 'A', fontsize = 16); fig.text(0.04, 0.70, 'B', fontsize = 16)
fig.text(0.04, 0.49, 'C', fontsize = 16); fig.text(0.04, 0.27, 'D', fontsize = 16)

fig.savefig('%s/%s.%s' % (DIR_WRITE, 'Figure_3_new', filetype), dpi = dpi, 
            facecolor='w', edgecolor='w', orientation='portrait', papertype=None, format = filetype,
            transparent=False, bbox_inches=None, pad_inches=0.1, frameon=True);